# dwtls: Discrete Wavelet Transform LayerS
This library provides downsampling (DS) layers using discrete wavelet transforms (DWTs), which we call DWT layers.
Conventional DS layers lack either antialiasing filters and the perfect reconstruction property, so downsampled features are aliased and entire information of input features are not preserved.
By contrast, DWT layers have antialiasing filters and the perfect reconstruction property, which enables us to overcome the two problems.

In this library, the DWT layer and its extensions are implemented as below:
- DWT layers with fixed wavelets (Haar, CDF22, CDF26, CDF15, and DD4 wavelets)
- Trainable DWT (TDWT) layers
- Weight-normalized trainable DWT (WN-TDWT) layers

## DWT layers with fixed wavelets
The DWT layer (including its extensions) is implemeted as a subclass of `torch.nn.Module` provided by PyTorch, so we can easily use it in PyTorch-based scripts. Also, this layer is differentiable.

In [1]:
import torch
import dwtls

In [2]:
dwt_layer = dwtls.DWT(wavelet="haar")

In [3]:
feature = torch.normal(0.0, 1.0, size=(1,1,20)).float()
output_feature = dwt_layer(feature)
print('Input:', feature)
print("Output:", output_feature)

Input: tensor([[[-0.5404,  1.9763,  0.4884,  0.6151, -0.6700, -1.6151, -0.7866,
          -0.1171,  0.0226,  1.9563,  0.9426, -0.0708,  0.2483,  0.3700,
          -0.0984, -0.6903,  0.0558, -0.2917,  3.4382, -0.9156]]])
Output: tensor([[[ 1.7795,  0.0896, -0.6683,  0.4734,  1.3674, -0.7165,  0.0860,
          -0.4185, -0.2457, -3.0786],
         [ 1.0153,  0.7803, -1.6158, -0.6390,  1.3993,  0.6165,  0.4372,
          -0.5577, -0.1668,  1.7837]]])


## TDWT layer
The TDWT layer has trainable wavelets (precisely, predict and update filters of lifting scheme).

For example, we can define the TDWT layer having a pair of the prediction and update filters initialized with Haar wavelet.

In [4]:
tdwt_layer = dwtls.MultiStageLWT([
    dict(predict_ksize=3, update_ksize=3, 
         requires_grad={"predict": True, "update": True}, 
         initial_values={"predict": [0,1,0], "update": [0,0.5,0]})
])

The `tdwt_layer._predict_weight` and `tdwt_layer._update_weight` of this layer are trainable jointly with other DNN components.

We show three structures of the trainable DWT layers used in our music source separation paper [1].  

[1] Tomohiko Nakamura, Shihori Kozuka, and Hiroshi Saruwatari, “Time-Domain Audio Source Separation with Neural Networks Based on Multiresolution Analysis,” IEEE/ACM Transactions on Audio, Speech, and Language Processing, vol. 29, pp. 1687–1701, Apr. 2021. [pdf](https://doi.org/10.1109/TASLP.2021.3072496), [demo](https://tomohikonakamura.github.io/Tomohiko-Nakamura/demo/MRDLA/)


In [5]:
# Type A
tdwt_layer = dwtls.MultiStageLWT([
    dict(predict_ksize=3, update_ksize=3, 
         requires_grad={"predict": True, "update": True}, 
         initial_values={"predict": [0,1,0], "update": [0,0.5,0]})
])

In [7]:
# Type B
tdwt_layer = dwtls.MultiStageLWT([
    dict(predict_ksize=1, update_ksize=1, 
         requires_grad={"predict": False, "update": False}, 
         initial_values={"predict": [1], "update": [0.5]}),
    dict(predict_ksize=3, update_ksize=3, 
         requires_grad={"predict": True, "update": True}, 
         initial_values={"predict": [0,0,0], "update": [0,0,0]})
])

In [8]:
# Type C
tdwt_layer = dwtls.MultiStageLWT([
    dict(predict_ksize=3, update_ksize=3, 
         requires_grad={"predict": True, "update": True}, 
         initial_values={"predict": [0,1,0], "update": [0,0.5,0]}),
    dict(predict_ksize=3, update_ksize=3, 
         requires_grad={"predict": True, "update": True}, 
         initial_values={"predict": [0,0,0], "update": [0,0,0]})
])

## WN-TDWT layer
The TDWT layer can be incorporated into many types of DNNs, but such straightforward extension does not guarantee that it has anti-aliasing filters, while it has the perfect reconstruction property owing to the lifting scheme.

The WN-TDWT layer is developed to overcome this problem. It has both properties owing to adequate normalization of the prediction and update filter coefficients.

In [9]:
# Type A
tdwt_layer = dwtls.WeightNormalizedMultiStageLWT([
    dict(predict_ksize=3, update_ksize=3, 
         requires_grad={"predict": True, "update": True}, 
         initial_values={"predict": [0,1,0], "update": [0,0.5,0]})
])

The WN-TDWT layer can be used in the same way as the TDWT layer.